In [1]:
import os

db_path = '../../data/predix_db/GTEx-V6p-HapMap-2016-09-08/'
fns = [f for f in os.listdir(db_path) if f[-3:] == '.db']
tissues = [f[3:-7] for f in fns]

In [2]:
import numpy as np
import pandas as pd
import sqlite3
import pickle

save_path = '../processed_data/model_genes/var/'

for tissue in tissues:
    
    df = pd.read_csv(os.path.join(db_path, 'TW_%s.txt.gz' % tissue), sep = ' ')

    conn = sqlite3.connect(os.path.join(db_path, 'TW_%s_0.5.db' % tissue))

    id2gene = [g for g in df.GENE]

    gene2var = {}


    res = []
    for g0, df0 in df.groupby('GENE'):

        id2rsid = sorted(set(df0.RSID1) | set(df0.RSID2))
        rsid2id = {r:i for i, r in enumerate(id2rsid)}

        cov_rsid = np.zeros((len(id2rsid), len(id2rsid)))
        for r1, r2, v in zip(df0.RSID1, df0.RSID2, df0.VALUE):
            cov_rsid[rsid2id[r1], rsid2id[r2]] = v
            cov_rsid[rsid2id[r2], rsid2id[r1]] = v


        dfg = pd.read_sql_query('SELECT * FROM weights WHERE gene=\'%s\'' % g0, conn)

        vec_w = np.zeros(len(id2rsid))
        for r, w in zip(dfg.rsid, dfg.weight):
            vec_w[rsid2id[r]] = w


        gene2var[g0] = vec_w.dot(cov_rsid).dot(vec_w)

    with open(os.path.join(save_path, '%s-gene2var.p' % tissue.replace('-', '_') ), 'wb') as writer:
        pickle.dump(gene2var, writer)
